In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rapidfuzz import process

import csv
import string
import pandasql as psql
from sqlalchemy import create_engine
import sqlite3

In [2]:
BD = pd.read_csv('COVID-19 Activity.csv', low_memory = False)
BD.set_index('index', inplace = True)

In [3]:
BD['CONTINENT_NAME'].fillna(value = 'Others', inplace = True)
BD['COUNTY_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['PROVINCE_STATE_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['CONTINENT_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['COUNTY_FIPS_NUMBER'].fillna(value = '0.0', inplace = True)

In [4]:
Cont_Uni_ID = pd.DataFrame({'CONTINENT_NAME': BD['CONTINENT_NAME'].unique()})
Cont_Uni_ID['Cont_ID'] = Cont_Uni_ID['CONTINENT_NAME'].str[:2].str.upper() + pd.RangeIndex(start = 10, stop = 16, step = 1).astype(str)

Cont_Uni_ID.set_index(['CONTINENT_NAME'], inplace = True)

Prov_Uni_ID = pd.DataFrame({'PROVINCE_STATE_NAME' : BD['PROVINCE_STATE_NAME'].unique()})
Prov_Uni_ID['PV_ID'] = Prov_Uni_ID['PROVINCE_STATE_NAME'].str[:2].str.upper() + "_" + pd.RangeIndex(len(BD['PROVINCE_STATE_NAME'].unique())).astype(str)

Prov_Uni_ID.set_index('PROVINCE_STATE_NAME', inplace = True)

Coun_Uni_ID = pd.DataFrame({'COUNTY_NAME' : BD['COUNTY_NAME'].unique()})
Coun_Uni_ID['COTY_ID'] = Coun_Uni_ID['COUNTY_NAME'].str[:2].str.upper() + pd.RangeIndex(len(BD['COUNTY_NAME'].unique())).astype(str)

Coun_Uni_ID.set_index('COUNTY_NAME', inplace = True)

DT_Uni_ID = pd.DataFrame({'DATA_SOURCE_NAME' : BD['DATA_SOURCE_NAME'].unique()})
DT_Uni_ID['DS_ID'] = DT_Uni_ID['DATA_SOURCE_NAME'].str[:2].str.upper() + pd.RangeIndex(len(BD['DATA_SOURCE_NAME'].unique())).astype(str)

DT_Uni_ID.set_index('DATA_SOURCE_NAME', inplace = True)

Co_Uni_ID = pd.DataFrame({'COUNTRY' : BD['COUNTRY'].unique()})
Co_Uni_ID['CTR_ID'] = Co_Uni_ID['COUNTRY'].str[:2].str.upper() + pd.RangeIndex(len(BD['COUNTRY'].unique())).astype(str)

Co_Uni_ID.set_index('COUNTRY', inplace = True)

In [5]:
Big_DS = pd.merge(BD, Cont_Uni_ID, how = 'left', on = 'CONTINENT_NAME')
Big_DS = pd.merge(Big_DS, Prov_Uni_ID, how = 'left', on = 'PROVINCE_STATE_NAME')
Big_DS = pd.merge(Big_DS, Coun_Uni_ID, how = 'left', on = 'COUNTY_NAME')
Big_DS = pd.merge(Big_DS, DT_Uni_ID, how = 'left', on = 'DATA_SOURCE_NAME')
Big_DS = pd.merge(Big_DS, Co_Uni_ID, how = 'left', on = 'COUNTRY')

In [6]:
Coun_alpha1_Uni_ID = pd.DataFrame({'CN_ALP_ID' : BD['COUNTRY_ALPHA_3_CODE'].unique()})
Coun_alpha2_Uni_ID = pd.DataFrame({'CN_ALP_ID' : BD['COUNTRY_ALPHA_2_CODE'].unique()})

Coun_alpha_Uni_ID = Coun_alpha1_Uni_ID + "_" + Coun_alpha2_Uni_ID

In [7]:
Big_DS['BD_ID'] = Big_DS['Cont_ID'] + Big_DS['PV_ID'] + "_" + Big_DS['COTY_ID'] + Big_DS['DS_ID'] + "_" + Big_DS['CTR_ID'] + "_" + Big_DS['COUNTRY_ALPHA_2_CODE'].str[:2].str.upper()  + "-" + Big_DS['COUNTRY_ALPHA_3_CODE'].str[:2].str.upper() 
Big_DS.set_index('BD_ID', inplace = True)
Big_DS.reindex(sorted(Big_DS.columns), axis= 1)

,CONTINENT_NAME,COTY_ID,COUNTRY,COUNTRY_ALPHA_2_CODE,COUNTRY_ALPHA_3_CODE,COUNTY_FIPS_NUMBER,COUNTY_NAME,CTR_ID,Cont_ID,DATA_SOURCE_NAME,DS_ID,PEOPLE_DEATH_COUNT,PEOPLE_DEATH_NEW_COUNT,PEOPLE_POSITIVE_CASES_COUNT,PEOPLE_POSITIVE_NEW_CASES_COUNT,PROVINCE_STATE_NAME,PV_ID,REPORT_DATE
BD_ID,,,,,,,,,,,,,,,,,,
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,1,19088,11,Ohio,OH_0,23-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19099,11,Ohio,OH_0,24-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19105,6,Ohio,OH_0,25-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19120,15,Ohio,OH_0,26-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19143,23,Ohio,OH_0,27-07-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,7,California,CA_26,11-03-2022
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,0,California,CA_26,12-03-2022
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,0,California,CA_26,13-03-2022


In [8]:
Big_DS['Day'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.day
Big_DS['Week'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.isocalendar().week
Big_DS['Month'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.month
Big_DS['Year'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.year
Big_DS['Quarter'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.quarter

Big_DS.fillna(value = '0.0', inplace = True)

In [9]:
Am_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'America']
Af_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Africa']
As_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Asia']
Ep_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Europe']
On_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Oceania']
Ot_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Others']

In [10]:
Big_DS.to_csv('Big_DS.csv', index = True)
Am_Data.to_csv('Am_Data.csv', index = True)
Af_Data.to_csv('Af_Data.csv', index = True)
As_Data.to_csv('As_Data.csv', index = True)
Ep_Data.to_csv('Ep_Data.csv', index = True)
On_Data.to_csv('On_Data.csv', index = True)
Ot_Data.to_csv('Ot_Data.csv', index = True)

In [11]:
Big_DS['DPC_Ratio'] = round((Big_DS['PEOPLE_DEATH_COUNT'] / Big_DS['PEOPLE_POSITIVE_CASES_COUNT'] * 100), 6)
Big_DS['New_DPC_Ratio'] = round((Big_DS['PEOPLE_DEATH_NEW_COUNT'] / Big_DS['PEOPLE_POSITIVE_NEW_CASES_COUNT'] * 100), 6)
Big_DS['CC_Ratio'] = round((Big_DS['PEOPLE_POSITIVE_NEW_CASES_COUNT'] / Big_DS['PEOPLE_POSITIVE_CASES_COUNT'] * 100), 6)
Big_DS['NDCC_Ratio'] = round((Big_DS['PEOPLE_DEATH_NEW_COUNT'] / Big_DS['PEOPLE_DEATH_COUNT'] * 100), 6)

In [12]:
Big_DS['New_DPC_Ratio'].fillna(value = '0.0', inplace = True)
Big_DS['DPC_Ratio'].fillna(value = '0.0', inplace = True)
Big_DS['CC_Ratio'].fillna(value = '0.0', inplace = True)
Big_DS['NDCC_Ratio'].fillna(value = '0.0', inplace = True)

In [13]:
conn = sqlite3.connect(':memory:')

Big_DS.to_sql('Big_DS', conn, index = True, if_exists = 'replace')

1048575

In [14]:
Big_DS['Year'].unique()

array([2021, 2022, 2020])

In [15]:
Intro_Q = """SELECT * FROM Big_DS """

resIQ = pd.read_sql(Intro_Q, conn)
resIQ

,BD_ID,PEOPLE_POSITIVE_CASES_COUNT,COUNTY_NAME,PROVINCE_STATE_NAME,REPORT_DATE,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTY_FIPS_NUMBER,COUNTRY_ALPHA_3_CODE,...,CTR_ID,Day,Week,Month,Year,Quarter,DPC_Ratio,New_DPC_Ratio,CC_Ratio,NDCC_Ratio
0,AM10OH_0_DE0NE0_UN0_US-US,19088,Delaware,Ohio,23-07-2021,America,New York Times,1,39041.0,USA,...,UN0,23,29,7,2021,3,0.722967,9.090909,0.057628,0.724638
1,AM10OH_0_DE0NE0_UN0_US-US,19099,Delaware,Ohio,24-07-2021,America,New York Times,0,39041.0,USA,...,UN0,24,29,7,2021,3,0.722551,0.0,0.057595,0.0
2,AM10OH_0_DE0NE0_UN0_US-US,19105,Delaware,Ohio,25-07-2021,America,New York Times,0,39041.0,USA,...,UN0,25,29,7,2021,3,0.722324,0.0,0.031405,0.0
3,AM10OH_0_DE0NE0_UN0_US-US,19120,Delaware,Ohio,26-07-2021,America,New York Times,0,39041.0,USA,...,UN0,26,30,7,2021,3,0.721757,0.0,0.078452,0.0
4,AM10OH_0_DE0NE0_UN0_US-US,19143,Delaware,Ohio,27-07-2021,America,New York Times,0,39041.0,USA,...,UN0,27,30,7,2021,3,0.72089,0.0,0.120148,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,11-03-2022,America,New York Times,0,6007.0,USA,...,UN0,11,10,3,2022,1,0.964565,0.0,0.017768,0.0
1048571,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,12-03-2022,America,New York Times,0,6007.0,USA,...,UN0,12,10,3,2022,1,0.964565,0.0,0.0,0.0
1048572,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,13-03-2022,America,New York Times,0,6007.0,USA,...,UN0,13,10,3,2022,1,0.964565,0.0,0.0,0.0
1048573,AM10CA_26_BU861NE0_UN0_US-US,39428,Butte,California,14-03-2022,America,New York Times,4,6007.0,USA,...,UN0,14,11,3,2022,1,0.973927,12.5,0.081161,1.041667


In [16]:
#Extra
E1 = """
SELECT 
        CONTINENT_NAME,
        COUNTRY,
        ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 5) AS DPC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 5) DESC) AS Continent_CC_Rank,
        ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) AS New_DPC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) DESC) AS Continent_NCC_Rank
    FROM 
        Big_DS
    GROUP BY
        COUNTRY
    ORDER BY 
        Continent_CC_Rank ASC, Continent_NCC_Rank ASC
"""

resE1 = pd.read_sql(E1, conn)
resE1

,CONTINENT_NAME,COUNTRY,DPC_Ratio,Continent_CC_Rank,New_DPC_Ratio,Continent_NCC_Rank
0,Africa,Sudan,0.07016,1,0.07938,1
1,America,Peru,0.08452,1,0.05970,1
2,Asia,Yemen,0.20254,1,0.18184,1
3,Europe,Bosnia and Herzegovina,0.04273,1,0.04181,1
4,Oceania,Papua New Guinea,0.01369,1,0.01483,1
...,...,...,...,...,...,...
222,America,Falkland Islands (Malvinas),0.00000,53,0.00000,53
223,Europe,Channel Islands,NaN,53,NaN,53
224,Africa,Seychelles,0.00449,54,0.00391,54
225,Africa,Reunion,0.00359,55,0.00193,55


In [17]:
Q1_PartA= """
WITH
    Rank_Checking AS (
        SELECT 
            CONTINENT_NAME, 
            COUNTRY,
            SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
            SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
            ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio, 
            RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6)  DESC) AS Country_CC_Rank,            
            SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
            SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
            ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS New_DPC_Ratio,
            RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS Country_NCC_Rank
        FROM 
            Big_DS 
        GROUP BY
            CONTINENT_NAME,
            COUNTRY 
        ORDER BY 
            Country_CC_Rank ASC, 
            Country_NCC_Rank ASC
    )
SELECT
    *
FROM
    Rank_Checking
WHERE
    Country_NCC_Rank = 23
"""

res1A = pd.read_sql(Q1_PartA, conn)
res1A

,CONTINENT_NAME,COUNTRY,Cases_Count,Death_Count,DPC_Ratio,Country_CC_Rank,New_Cases_Count,New_Death_Count,New_DPC_Ratio,Country_NCC_Rank
0,Europe,United Kingdom,4637262360,77403359,0.016692,16,22038340,174912,0.007937,23
1,Africa,Angola,27716141,629994,0.022730,20,99287,1900,0.019136,23
2,Asia,Jordan,447932295,5026901,0.011222,22,1695745,14064,0.008294,23
3,America,Saint Vincent and the Grenadines,1938382,20585,0.010620,36,8419,106,0.012591,23


In [18]:
Q1_PartB = """
WITH 
    Rank_Checking AS (
        SELECT 
            CONTINENT_NAME, 
            COUNTRY, 
            PROVINCE_STATE_NAME,
            SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
            SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
            ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio, 
            RANK() OVER(PARTITION BY PROVINCE_STATE_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS PROVINCE_STATE_CC_Rank,            
            SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
            SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
            ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6)  AS New_DPC_Ratio,
            RANK() OVER(PARTITION BY PROVINCE_STATE_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS PROVINCE_STATE_NCC_Rank
        FROM 
            Big_DS 
        GROUP BY 
            COUNTRY, PROVINCE_STATE_NAME 
        ORDER BY 
            PROVINCE_STATE_CC_Rank ASC, 
            PROVINCE_STATE_NCC_Rank ASC
    )
SELECT
    *
FROM
    Rank_Checking
WHERE
    PROVINCE_STATE_CC_Rank = 13
"""

res1B = pd.read_sql(Q1_PartB, conn)
res1B

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,Cases_Count,Death_Count,DPC_Ratio,PROVINCE_STATE_CC_Rank,New_Cases_Count,New_Death_Count,New_DPC_Ratio,PROVINCE_STATE_NCC_Rank
0,Europe,Bulgaria,Not Provided,293028335,11054140,0.037724,13,1156466,36911,0.031917,13


In [19]:
Q1_PartC = """
SELECT 
    CONTINENT_NAME, 
    COUNTRY, 
    PROVINCE_STATE_NAME, 
    COUNTY_NAME,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio, 
    RANK() OVER(PARTITION BY COUNTY_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS County_CC_Rank,            
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6)  AS New_DPC_Ratio,
    RANK() OVER(PARTITION BY COUNTY_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS County_NCC_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY, 
    PROVINCE_STATE_NAME, 
    COUNTY_NAME
ORDER BY 
    County_CC_Rank ASC, 
    County_NCC_Rank ASC
"""

res1C = pd.read_sql(Q1_PartC, conn)
res1C.dropna()

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,COUNTY_NAME,Cases_Count,Death_Count,DPC_Ratio,County_CC_Rank,New_Cases_Count,New_Death_Count,New_DPC_Ratio,County_NCC_Rank
2,America,United States,Virginia,Accomack,2208006,34610,0.015675,1,6984,106,0.015178,1
4,America,United States,Iowa,Adair,550945,18071,0.032800,1,1587,50,0.031506,1
5,America,United States,Washington,Adams,1601360,15013,0.009375,1,4919,38,0.007725,1
8,America,United States,Puerto Rico,Aguada,1412974,0,0.000000,1,6794,0,0.000000,1
10,America,United States,Puerto Rico,Aguas Buenas,1002708,0,0.000000,1,4253,0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3551,Europe,Holy See,Not Provided,Not Provided,17701,0,0.000000,265,29,0,0.000000,264
3552,Oceania,Marshall Islands,Not Provided,Not Provided,2605,0,0.000000,265,17,0,0.000000,264
3553,Others,Micronesia,Not Provided,Not Provided,480,0,0.000000,265,5,0,0.000000,264
3554,Others,Summer Olympics 2020,Not Provided,Not Provided,227078,0,0.000000,265,865,0,0.000000,264


In [20]:
Q1_PartD = """
SELECT 
    CONTINENT_NAME, 
    COUNTRY, 
    PROVINCE_STATE_NAME, 
    COUNTY_NAME,
    COUNTY_FIPS_NUMBER,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio, 
    RANK() OVER(PARTITION BY COUNTY_FIPS_NUMBER ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS County_CC_Rank,            
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6)  AS New_DPC_Ratio,
    RANK() OVER(PARTITION BY COUNTY_FIPS_NUMBER ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS County_NCC_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY, 
    PROVINCE_STATE_NAME, 
    COUNTY_NAME,
    COUNTY_FIPS_NUMBER
ORDER BY 
    County_CC_Rank ASC, 
    County_NCC_Rank ASC
"""

res1D = pd.read_sql(Q1_PartD, conn)
res1D.dropna()

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,COUNTY_NAME,COUNTY_FIPS_NUMBER,Cases_Count,Death_Count,DPC_Ratio,County_CC_Rank,New_Cases_Count,New_Death_Count,New_DPC_Ratio,County_NCC_Rank
3,America,United States,Alabama,Autauga,1001.0,4998534,70129,0.014030,1,15827,216,0.013648,1
4,America,United States,Alabama,Baldwin,1003.0,17154512,225395,0.013139,1,55643,680,0.012221,1
9,America,United States,Alabama,Butler,1013.0,1696233,49856,0.029392,1,5064,129,0.025474,1
10,America,United States,Alabama,Calhoun,1015.0,10452288,210248,0.020115,1,32399,626,0.019322,1
14,America,United States,Alabama,Choctaw,1023.0,542023,16143,0.029783,1,2050,36,0.017561,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,Europe,Holy See,Not Provided,Not Provided,0.0,17701,0,0.000000,286,29,0,0.000000,274
3516,Oceania,Marshall Islands,Not Provided,Not Provided,0.0,2605,0,0.000000,286,17,0,0.000000,274
3517,Others,Micronesia,Not Provided,Not Provided,0.0,480,0,0.000000,286,5,0,0.000000,274
3518,Others,Summer Olympics 2020,Not Provided,Not Provided,0.0,227078,0,0.000000,286,865,0,0.000000,274


In [21]:
Q2_PartA= """
SELECT 
    CONTINENT_NAME, 
    COUNTRY,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS Cases_Ratio, 
    RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6)  DESC) AS Country_CCR_Rank,            
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) AS Death_Ratio,
    RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS Country_DR_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY 
ORDER BY 
    Country_CCR_Rank ASC, 
    Country_DR_Rank ASC
"""

res2A = pd.read_sql(Q2_PartA, conn)
res2A.dropna()

,CONTINENT_NAME,COUNTRY,Cases_Count,New_Cases_Count,Cases_Ratio,Country_CCR_Rank,Death_Count,New_Death_Count,Death_Ratio,Country_DR_Rank
0,Africa,Sudan,23088035,62117,371.686253,1,1619890,4931,328.511458,1
1,America,Peru,1217148442,3564558,341.458448,1,102867709,212810,483.378173,1
2,Asia,Yemen,4141371,11818,350.429091,1,838774,2149,390.308981,1
3,Europe,Bosnia and Herzegovina,124155885,377087,329.249974,1,5304763,15765,336.489883,1
4,Oceania,Papua New Guinea,10725712,43817,244.784262,1,146885,650,225.976923,1
...,...,...,...,...,...,...,...,...,...,...
219,America,Saint Pierre and Miquelon,175947,2701,65.141429,52,88,1,88.000000,52
221,Africa,Mauritius,23028633,220344,104.512185,53,149639,990,151.150505,53
224,Africa,Seychelles,9389138,42502,220.910498,54,42150,166,253.915663,54
225,Africa,Reunion,44466022,387184,114.844678,55,159737,749,213.267023,55


In [22]:
Q2_PartB= """
SELECT 
    CONTINENT_NAME, 
    COUNTRY,
    PROVINCE_STATE_NAME,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS Cases_Ratio, 
    RANK() OVER(PARTITION BY PROVINCE_STATE_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6)  DESC) AS Country_CCR_Rank,            
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) AS Death_Ratio,
    RANK() OVER(PARTITION BY PROVINCE_STATE_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS Country_DR_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY, 
    PROVINCE_STATE_NAME
ORDER BY 
    Country_CCR_Rank ASC, 
    Country_DR_Rank ASC
"""

res2B = pd.read_sql(Q2_PartB, conn)
res2B.dropna()

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,Cases_Count,New_Cases_Count,Cases_Ratio,Country_CCR_Rank,Death_Count,New_Death_Count,Death_Ratio,Country_DR_Rank
0,America,United States,Alabama,162115225,514416,315.144212,1,2774875,7641,363.156001,1
1,America,United States,Alaska,49604778,197291,251.429503,1,264196,1034,255.508704,1
2,America,Canada,Alberta,149872547,565052,265.236734,1,1387420,4252,326.298213,1
3,America,United States,American Samoa,187516,5517,33.988762,1,268,12,22.333333,1
4,Asia,China,Anhui,815820,1065,766.028169,1,4853,6,808.833333,1
...,...,...,...,...,...,...,...,...,...,...,...
323,Others,Tonga,Not Provided,340910,10196,33.435661,211,345,11,31.363636,207
324,Oceania,New Zealand,Not Provided,40258581,930969,43.243740,212,39451,702,56.198006,210
325,Asia,Bhutan,Not Provided,3050626,59054,51.658245,213,1620,21,77.142857,213
326,America,Saint Pierre and Miquelon,Not Provided,175947,2701,65.141429,214,88,1,88.000000,212


In [23]:
Q2_PartC= """
SELECT 
    CONTINENT_NAME, 
    COUNTRY,
    PROVINCE_STATE_NAME,
    COUNTY_NAME,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS Cases_Ratio, 
    RANK() OVER(PARTITION BY COUNTY_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6)  DESC) AS Country_CCR_Rank,            
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) AS Death_Ratio,
    RANK() OVER(PARTITION BY COUNTY_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS Country_DR_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY, 
    PROVINCE_STATE_NAME,
    COUNTY_NAME
ORDER BY 
    Country_CCR_Rank ASC, 
    Country_DR_Rank ASC
"""

res2C = pd.read_sql(Q2_PartC, conn)
res2C.dropna()

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,COUNTY_NAME,Cases_Count,New_Cases_Count,Cases_Ratio,Country_CCR_Rank,Death_Count,New_Death_Count,Death_Ratio,Country_DR_Rank
2,America,United States,Virginia,Accomack,2208006,6984,316.152062,1,34610,106,326.509434,1
4,America,United States,Iowa,Adair,550945,1587,347.161311,1,18071,50,361.420000,1
5,America,United States,Washington,Adams,1601360,4919,325.545843,1,15013,38,395.078947,1
12,America,United States,South Carolina,Aiken,12623718,41569,303.681060,1,154189,437,352.835240,1
13,America,United States,Minnesota,Aitkin,935629,3003,311.564769,1,22634,62,365.064516,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3536,Asia,China,Zhejiang,Not Provided,1229102,3109,395.336764,260,800,1,800.000000,261
3537,Oceania,Australia,Northern Territory,Not Provided,4244732,63289,67.069033,261,2445,43,56.860465,252
3538,Asia,Bhutan,Not Provided,Not Provided,3050626,59054,51.658245,262,1620,21,77.142857,259
3539,America,Saint Pierre and Miquelon,Not Provided,Not Provided,175947,2701,65.141429,263,88,1,88.000000,257


In [24]:
Q2_PartD= """
SELECT 
    CONTINENT_NAME, 
    COUNTRY,
    PROVINCE_STATE_NAME,
    COUNTY_NAME,
    COUNTY_FIPS_NUMBER,
    SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
    SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count, 
    ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS Cases_Ratio, 
    RANK() OVER(PARTITION BY COUNTY_FIPS_NUMBER ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6)  DESC) AS Country_CCR_Rank,            
    SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
    SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count, 
    ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) AS Death_Ratio,
    RANK() OVER(PARTITION BY COUNTY_FIPS_NUMBER ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS Country_DR_Rank
FROM 
    Big_DS 
GROUP BY 
    COUNTRY, 
    PROVINCE_STATE_NAME,
    COUNTY_NAME,
    COUNTY_FIPS_NUMBER
ORDER BY 
    Country_CCR_Rank ASC, 
    Country_DR_Rank ASC
"""

res2D = pd.read_sql(Q2_PartD, conn)
res2D.dropna()

,CONTINENT_NAME,COUNTRY,PROVINCE_STATE_NAME,COUNTY_NAME,COUNTY_FIPS_NUMBER,Cases_Count,New_Cases_Count,Cases_Ratio,Country_CCR_Rank,Death_Count,New_Death_Count,Death_Ratio,Country_DR_Rank
3,America,United States,Alabama,Autauga,1001.0,4998534,15827,315.823213,1,70129,216,324.671296,1
4,America,United States,Alabama,Baldwin,1003.0,17154512,55643,308.295958,1,225395,680,331.463235,1
9,America,United States,Alabama,Butler,1013.0,1696233,5064,334.959123,1,49856,129,386.480620,1
10,America,United States,Alabama,Calhoun,1015.0,10452288,32399,322.611439,1,210248,626,335.859425,1
14,America,United States,Alabama,Choctaw,1023.0,542023,2050,264.401463,1,16143,36,448.416667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3497,Asia,Bhutan,Not Provided,Not Provided,0.0,3050626,59054,51.658245,278,1620,21,77.142857,268
3498,America,Saint Pierre and Miquelon,Not Provided,Not Provided,0.0,175947,2701,65.141429,279,88,1,88.000000,266
3499,America,United States,Oklahoma,Unknown,0.0,2729027,628,4345.584395,280,732,4,183.000000,168
3500,America,United States,Massachusetts,Unknown,0.0,33269157,140783,236.315159,281,8819,22,400.863636,272


In [25]:
"""
Test_Query = '''
WITH Continent_cases AS (
    SELECT 
        CONTINENT_NAME,
        SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, 
        SUM(PEOPLE_DEATH_COUNT) AS Death_Count, 
        ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,
    FROM 
        Big_DS
    GROUP BY 
        CONTINENT_NAME
),
Continent_Ranks AS (
    SELECT 
        CONTINENT_NAME,
        Cases_Count,
        Death_Count,
        DPC_Ratio,
        RANK() OVER(ORDER BY DPC_Ratio DESC) AS Continent_Rank
    FROM 
        Continent_cases
    GROUP BY CONTINENT_NAME
),
Country_Ranks AS (
    SELECT 
        ctry.CONTINENT_NAME,
        ctry.COUNTRY,
        ROUND(SUM(ctry.PEOPLE_DEATH_COUNT) * 1.0 / SUM(ctry.PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,
        RANK() OVER(PARTITION BY ctry.COUNTRY ORDER BY ROUND(SUM(ctry.PEOPLE_DEATH_COUNT) * 1.0 / SUM(ctry.PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Country_Rank
    FROM
        Big_DS ctry
    GROUP BY
        ctry.CONTINENT_NAME,
        ctry.COUNTRY
)
SELECT
    contr.CONTINENT_NAME,
    contr.Cases_Count,
    contr.Death_Count,
    contr.DPC_Ratio,
    contr.Continent_Rank,
    cntry.COUNTRY,
    cntry.Country_Rank
FROM
    Continent_Ranks contr
JOIN
    Country_Ranks cntry
ON
    cntry.CONTINENT_NAME = contr.CONTINENT_NAME
ORDER BY 
    cntry.COUNTRY,
    contr.Continent_Rank, 
    cntry.Country_Rank
'''

resTQ = psql.sqldf(Test_Query, locals())
resTQ
"""

"\nTest_Query = '''\nWITH Continent_cases AS (\n    SELECT \n        CONTINENT_NAME,\n        SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count, \n        SUM(PEOPLE_DEATH_COUNT) AS Death_Count, \n        ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,\n    FROM \n        Big_DS\n    GROUP BY \n        CONTINENT_NAME\n),\nContinent_Ranks AS (\n    SELECT \n        CONTINENT_NAME,\n        Cases_Count,\n        Death_Count,\n        DPC_Ratio,\n        RANK() OVER(ORDER BY DPC_Ratio DESC) AS Continent_Rank\n    FROM \n        Continent_cases\n    GROUP BY CONTINENT_NAME\n),\nCountry_Ranks AS (\n    SELECT \n        ctry.CONTINENT_NAME,\n        ctry.COUNTRY,\n        ROUND(SUM(ctry.PEOPLE_DEATH_COUNT) * 1.0 / SUM(ctry.PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,\n        RANK() OVER(PARTITION BY ctry.COUNTRY ORDER BY ROUND(SUM(ctry.PEOPLE_DEATH_COUNT) * 1.0 / SUM(ctry.PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Country_Rank\n    FROM\n        Big_D

In [26]:
Test_Query_1 = """
WITH 
ContinentRank AS (
    SELECT
        CONTINENT_NAME,
        SUM(DPC_Ratio) AS DPC_Ratio,
        SUM(New_DPC_Ratio) AS New_DPC_Ratio,
        RANK() OVER(ORDER BY SUM(DPC_Ratio) DESC) AS Continent_CC_Rank,
        RANK() OVER(ORDER BY SUM(New_DPC_Ratio) DESC) AS Continent_NCC_Rank
    FROM
        Big_DS 
    GROUP BY
        CONTINENT_NAME
),
CountryRank AS (
    SELECT
        CONTINENT_NAME,
        COUNTRY,
        SUM(DPC_Ratio) AS DPC_Ratio,
        SUM(New_DPC_Ratio) AS New_DPC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY SUM(DPC_Ratio) DESC) AS Country_CC_Rank,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY SUM(New_DPC_Ratio) DESC) AS Country_NCC_Rank
    FROM
        Big_DS
    GROUP BY 
        COUNTRY
)
SELECT
    cr.CONTINENT_NAME,
    cr.COUNTRY,
    cr.DPC_Ratio,
    cr.New_DPC_Ratio,
    cnr.Continent_CC_Rank,
    cnr.Continent_NCC_Rank,
    cr.Country_CC_Rank,
    cr.Country_NCC_Rank
FROM
    CountryRank cr
JOIN
    ContinentRank cnr 
ON
    cr.CONTINENT_NAME = cnr.CONTINENT_NAME
ORDER BY
    cr.CONTINENT_NAME,
    cnr.Continent_CC_Rank,
    cnr.Continent_NCC_Rank,
    cr.Country_CC_Rank,
    cr.Country_NCC_Rank
"""

resTQ1 = psql.sqldf(Test_Query_1, locals())
resTQ1

,CONTINENT_NAME,COUNTRY,DPC_Ratio,New_DPC_Ratio,Continent_CC_Rank,Continent_NCC_Rank,Country_CC_Rank,Country_NCC_Rank
0,Africa,Sudan,6453.837648,5616.263767,2,3,1,1
1,Africa,Egypt,4289.349086,4619.112091,2,3,2,2
2,Africa,Liberia,3887.223189,2248.446864,2,3,3,14
3,Africa,Chad,3464.926477,1675.069475,2,3,4,20
4,Africa,Niger,3260.214578,2407.886107,2,3,5,12
...,...,...,...,...,...,...,...,...
222,Others,Cook Islands,0.101950,0.000000,6,6,8,8
223,Others,Antarctica,0.000000,0.000000,6,6,9,8
224,Others,Micronesia,0.000000,0.000000,6,6,9,8
225,Others,Summer Olympics 2020,0.000000,0.000000,6,6,9,8


In [27]:
import math 

def SQRT(x) :
    
    if x is None :
        return None

    return math.sqrt(x)

conn.create_function("SQRT", 1, SQRT)

In [60]:
#Country Name is remaining
Q3_PartA1 = """
WITH 
Checker AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        ROUND(SUM(PEOPLE_DEATH_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Continent_CC_Rank,
        ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) AS New_DPC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) DESC) AS Continent_NCC_Rank
    FROM 
        Big_DS
    GROUP BY
        COUNTRY
    ORDER BY 
        Continent_CC_Rank ASC, 
        Continent_NCC_Rank ASC
)
SELECT
    CONTINENT_NAME,
    MAX(DPC_Ratio) AS MAX_old,
    AVG(DPC_Ratio) AS AVG_old,
    MIN(DPC_Ratio) AS MIN_old,
    AVG(DPC_Ratio * DPC_Ratio) - AVG(DPC_Ratio) * AVG(DPC_Ratio) AS VR_old,
    SQRT(AVG(DPC_Ratio * DPC_Ratio) - AVG(DPC_Ratio) * AVG(DPC_Ratio)) AS STD_old,
    MAX(New_DPC_Ratio) AS MAX_new,
    AVG(New_DPC_Ratio) AS AVG_new,
    MIN(New_DPC_Ratio) AS MIN_new,
    AVG(New_DPC_Ratio * New_DPC_Ratio) - AVG(New_DPC_Ratio) * AVG(New_DPC_Ratio) AS VR_new,
    SQRT(AVG(New_DPC_Ratio * New_DPC_Ratio) - AVG(New_DPC_Ratio) * AVG(New_DPC_Ratio)) AS STD_new
FROM
    Checker
GROUP BY
    CONTINENT_NAME
"""

res3A1 = pd.read_sql(Q3_PartA1, conn)
res3A1

,CONTINENT_NAME,MAX_old,AVG_old,MIN_old,VR_old,STD_old,MAX_new,AVG_new,MIN_new,VR_new,STD_new
0,Africa,7.016145,2.011420,0.147225,1.745918,1.321332,7.93825,1.841315,0.09500,1.751850,1.323575
1,America,8.451534,1.814071,0.000000,2.584602,1.607670,5.97017,1.443364,0.00000,1.494820,1.222628
2,Asia,20.253534,1.832660,0.053104,9.640614,3.104934,18.18413,1.533379,0.03556,7.875130,2.806266
3,Europe,4.272663,1.433272,0.000000,0.782034,0.884327,4.18073,1.010214,0.00000,0.752623,0.867538
4,Oceania,1.369466,0.658022,0.000000,0.264798,0.514585,1.48344,0.574910,0.00000,0.273331,0.522810
5,Others,1.469596,0.292728,0.000000,0.176902,0.420597,1.54185,0.220818,0.00000,0.174489,0.417719


In [70]:
Q3_PartA2 = """
WITH 
Checker AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) AS DD_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT) * 1.0 / SUM(PEOPLE_DEATH_NEW_COUNT), 6) DESC) AS Continent_DD_Rank,
        ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) AS CC_Ratio,
        RANK() OVER(PARTITION BY CONTINENT_NAME ORDER BY ROUND(SUM(PEOPLE_POSITIVE_CASES_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 5) DESC) AS Continent_CC_Rank
    FROM 
        Big_DS
    GROUP BY
        COUNTRY
    ORDER BY 
        Continent_DD_Rank ASC, 
        Continent_CC_Rank ASC
)
SELECT
    CONTINENT_NAME,
    MAX(DD_Ratio) AS MAX_old,
    AVG(DD_Ratio) AS AVG_old,
    MIN(DD_Ratio) AS MIN_old,
    ROUND((AVG(DD_Ratio * DD_Ratio) - AVG(DD_Ratio) * AVG(DD_Ratio)), 6) AS VR_old,
    ROUND(SQRT(AVG(DD_Ratio * DD_Ratio) - AVG(DD_Ratio) * AVG(DD_Ratio)), 6) AS STD_old,
    MAX(CC_Ratio) AS MAX_new,
    AVG(CC_Ratio) AS AVG_new,
    MIN(CC_Ratio) AS MIN_new,
    ROUND((AVG(CC_Ratio * CC_Ratio) - AVG(CC_Ratio) * AVG(CC_Ratio)), 6) AS VR_new,
    ROUND(SQRT(AVG(CC_Ratio * CC_Ratio) - AVG(CC_Ratio) * AVG(CC_Ratio)), 6) AS STD_new
FROM
    Checker
GROUP BY
    CONTINENT_NAME
"""

res3A2 = pd.read_sql(Q3_PartA2, conn)
res3A2

,CONTINENT_NAME,MAX_old,AVG_old,MIN_old,VR_old,STD_old,MAX_new,AVG_new,MIN_new,VR_new,STD_new
0,Africa,534.568000,361.026370,151.150505,5518.853736,74.288988,44230.22228,32328.683141,10451.21855,4.582323e+07,6769.285787
1,America,532.918803,306.956187,76.428571,10779.601975,103.824862,38145.72825,24346.333656,6514.14291,5.871619e+07,7662.649200
2,Asia,558.728000,322.217662,77.142857,11888.408519,109.033979,54449.88755,26524.153832,5063.64919,1.367482e+08,11693.937882
3,Europe,509.006536,345.930872,138.321429,7115.649441,84.354309,61037.93103,22553.201156,10131.77784,6.820466e+07,8258.611103
4,Oceania,299.026235,141.988279,13.736842,8923.214703,94.462769,25654.35506,12144.478470,1703.71143,8.179000e+07,9043.782494
5,Others,389.000000,184.381353,5.000000,26403.138169,162.490425,55650.00000,16718.436726,2584.17810,2.416057e+08,15543.669398


In [106]:
 Sampling_Qy1 = """
 WITH 
     Death_Cases_ratio AS (
         SELECT 
             CONTINENT_NAME,
             COUNTRY,
             Quarter,
             Year,
             SUM(PEOPLE_DEATH_COUNT) AS Death_Count,
             SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_COUNT)  * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DCP_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT)  * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Quarter_Rank,
             SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count,
             SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS New_DPC_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS New_Quarter_Rank
        FROM
            Big_DS
        GROUP BY
            Quarter,
            COUNTRY,
            CONTINENT_NAME
),
Old_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Death_Count,
        Cases_Count,
        DCP_Ratio,
        Quarter_Rank,
        New_Death_Count,
        New_Cases_Count,
        New_DPC_Ratio,
        New_Quarter_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 4 AND Year = 2020
    ORDER BY
        Quarter_Rank
    LIMIT 10
),
New_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Death_Count,
        Cases_Count,
        DCP_Ratio,
        Quarter_Rank,
        New_Death_Count,
        New_Cases_Count,
        New_DPC_Ratio,
        New_Quarter_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 4 AND Year = 2020
    ORDER BY
        New_Quarter_Rank
    LIMIT 10
)
SELECT
    DISTINCT *
FROM
    Old_Ranking
UNION ALL
SELECT
     DISTINCT *
FROM
    New_Ranking
"""

resQy1 = pd.read_sql(Sampling_Qy1, conn)
resQy1

,CONTINENT_NAME,COUNTRY,Quarter,Death_Count,Cases_Count,DCP_Ratio,Quarter_Rank,New_Death_Count,New_Cases_Count,New_DPC_Ratio,New_Quarter_Rank
0,Asia,Yemen,4,230157,1094638,0.210259,1,286,1124,0.254448,1
1,Oceania,Vanuatu,4,92,570,0.161404,2,0,4,0.000000,204
2,America,Peru,4,26567905,289802754,0.091676,3,14258,324895,0.043885,12
3,America,Mexico,4,35949845,449272200,0.080018,4,70082,998378,0.070196,4
4,Africa,Sudan,4,388545,5432609,0.071521,5,1061,20089,0.052815,8
5,America,Ecuador,4,4253144,64647115,0.065790,6,3576,114706,0.031175,27
6,Asia,Syria,4,277706,4780803,0.058088,7,1161,23307,0.049813,10
7,Africa,Egypt,4,2403258,42222714,0.056919,8,6122,115915,0.052815,8
8,Africa,Somalia,4,126904,2452466,0.051745,9,253,4678,0.054083,7
9,Africa,Liberia,4,33964,679499,0.049984,10,2,940,0.002128,194


In [104]:
Sampling_Qy2 = """
 WITH 
     Death_Cases_ratio AS (
         SELECT 
             CONTINENT_NAME,
             COUNTRY,
             Quarter,
             Year,
             SUM(PEOPLE_DEATH_COUNT) AS Death_Count,
             SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_COUNT)  * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DCP_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT)  * 1.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Quarter_Rank,
             SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count,
             SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS New_DPC_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 1.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS New_Quarter_Rank
        FROM
            Big_DS
        GROUP BY
            Quarter,
            COUNTRY,
            CONTINENT_NAME
),
Old_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Death_Count,
        Cases_Count,
        DCP_Ratio,
        Quarter_Rank,
        New_Death_Count,
        New_Cases_Count,
        New_DPC_Ratio,
        New_Quarter_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 1 AND Year = 2020
    ORDER BY
        Quarter_Rank ASC
    LIMIT 10
),
New_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Death_Count,
        Cases_Count,
        DCP_Ratio,
        Quarter_Rank,
        New_Death_Count,
        New_Cases_Count,
        New_DPC_Ratio,
        New_Quarter_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 1 AND Year = 2020
    ORDER BY
        New_Quarter_Rank ASC
    LIMIT 10
)
SELECT
    DISTINCT *
FROM
    Old_Ranking
UNION ALL
SELECT
     DISTINCT *
FROM
    New_Ranking
"""

resQy2 = pd.read_sql(Sampling_Qy2, conn)
resQy2

,CONTINENT_NAME,COUNTRY,Quarter,Death_Count,Cases_Count,DCP_Ratio,Quarter_Rank,New_Death_Count,New_Cases_Count,New_DPC_Ratio,New_Quarter_Rank
0,America,Peru,1,29023899,400120634,0.072538,1,60001,1784600,0.033622,6
1,Africa,Sudan,1,517616,7849919,0.065939,2,2173,21777,0.099784,1
2,Asia,Syria,1,360800,6114579,0.059007,3,800,12902,0.062006,2
3,Europe,Bosnia and Herzegovina,1,1786129,43330476,0.041221,4,4833,143188,0.033753,5
4,Africa,Liberia,1,33749,828049,0.040757,5,9,1378,0.006531,65
5,Europe,Bulgaria,1,3977676,111773627,0.035587,6,11203,532581,0.021035,13
6,Africa,Niger,1,41541,1179233,0.035227,7,118,3101,0.038052,4
7,Asia,Burma,1,2020396,63340531,0.031897,8,687,98572,0.006970,57
8,Europe,North Macedonia,1,1053835,34041919,0.030957,9,2548,127952,0.019914,14
9,Africa,Malawi,1,301079,9805387,0.030705,10,1190,37533,0.031705,7


In [199]:
Sampling_Qy3A = """
 WITH 
     Death_Cases_Ratio AS (
         SELECT 
             CONTINENT_NAME,
             COUNTRY,
             Week,
             Quarter,
             Year,
             SUM(PEOPLE_DEATH_COUNT) AS Death_Count,
             SUM(PEOPLE_POSITIVE_CASES_COUNT) AS Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_COUNT)  * 100.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) AS DPC_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT)  * 100.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Quarter_Rank,
             RANK() OVER(PARTITION BY Quarter, Year, Week ORDER BY ROUND(SUM(PEOPLE_DEATH_COUNT)  * 100.0 / SUM(PEOPLE_POSITIVE_CASES_COUNT), 6) DESC) AS Week_Rank,
             SUM(PEOPLE_DEATH_NEW_COUNT) AS New_Death_Count,
             SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT) AS New_Cases_Count,
             ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) AS New_DPC_Ratio,
             RANK() OVER(PARTITION BY Quarter, Year ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS New_Quarter_Rank,
             RANK() OVER(PARTITION BY Quarter, Year, Week ORDER BY ROUND(SUM(PEOPLE_DEATH_NEW_COUNT) * 100.0 / SUM(PEOPLE_POSITIVE_NEW_CASES_COUNT), 6) DESC) AS New_Week_Rank             
        FROM
            Big_DS
        GROUP BY
            Quarter,
            Week,
            COUNTRY,
            CONTINENT_NAME
),
Old_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Week,
        DPC_Ratio,
        Quarter_Rank,
        Week_Rank,
        New_DPC_Ratio,
        New_Quarter_Rank,
        New_Week_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 2 AND Year = 2020 AND Week = 1 OR Week = 4 AND Week_Rank < 3
    ORDER BY
        Quarter_Rank ASC
    LIMIT 10
),
New_Ranking AS (
    SELECT 
        CONTINENT_NAME,
        COUNTRY,
        Quarter,
        Week,
        DPC_Ratio,
        Quarter_Rank,
        Week_Rank,
        New_DPC_Ratio,
        New_Quarter_Rank,
        New_Week_Rank
    FROM
        Death_Cases_Ratio
    WHERE
        Quarter = 2 AND Year = 2020 AND Week = 1 OR Week = 4 AND New_Week_Rank < 3
    ORDER BY
        New_Quarter_Rank ASC
    LIMIT 10
)
SELECT
    DISTINCT *
FROM
    Old_Ranking
UNION ALL
SELECT
     DISTINCT *
FROM
    New_Ranking
ORDER BY
    Week_Rank,
    New_Week_Rank
"""

resQy3A = pd.read_sql(Sampling_Qy3A, conn)
resQy3A

,CONTINENT_NAME,COUNTRY,Quarter,Week,DPC_Ratio,Quarter_Rank,Week_Rank,New_DPC_Ratio,New_Quarter_Rank,New_Week_Rank
0,Asia,Yemen,1,4,20.241211,3,1,4.444444,54,3
1,Africa,Somalia,1,4,4.857565,41,1,0.000000,633,4
2,America,Peru,1,4,7.295216,29,1,1.575540,471,24
3,America,United States,1,4,1.299207,387,2,0.895591,285,1
4,America,United States,1,4,1.299207,387,2,0.895591,285,1
5,Africa,Sudan,1,4,6.070940,52,2,5.817728,102,4
6,Oceania,Vanuatu,1,4,12.500000,8,2,NaN,1113,86
7,Asia,Syria,1,4,5.970792,53,3,6.958763,78,2
8,Asia,Laos,1,4,0.404894,703,4,0.503637,398,2
9,Africa,Tanzania,1,4,2.384828,226,15,4.743833,51,2
